In [1]:
import os
import uuid
import zipfile
from os.path import join, dirname
from tempfile import gettempdir
from unittest import TestCase
import pandas as pd
import numpy as np

from aequilibrae import TrafficAssignment, TrafficClass, Graph, Project
from tests.data import siouxfalls_project


In [2]:
proj_path ='coming'
# Initialise project:
project = Project()
project.open(proj_path)
project.network.build_graphs()
car_graph = project.network.graphs["c"]  # type: Graph

car_graph.set_graph("free_flow_time")
car_graph.set_blocked_centroid_flows(False)
matrix = project.matrices.get_matrix("demand_omx")
matrix.computational_view()

# Extra data specific to ODME:
index = car_graph.nodes_to_indices
dims = matrix.matrix_view.shape
count_vol_cols = ["class", "link_id", "direction", "obs_volume"]

# Initial assignment parameters:
assignment = TrafficAssignment()
assignclass = TrafficClass("car", car_graph, matrix)
assignment.set_classes([assignclass])
assignment.set_vdf("BPR")
assignment.set_vdf_parameters({"alpha": 0.15, "beta": 4.0})
assignment.set_vdf_parameters({"alpha": "b", "beta": "power"})
assignment.set_capacity_field("capacity")
assignment.set_time_field("free_flow_time")
assignment.max_iter = 5
assignment.set_algorithm("msa")

C:\Users\penny\git\Aequilibrae\aequilibrae\paths\graph.py:146: FutureWarning: The behavior of array concatenation with empty entries is deprecated. In a future version, this will no longer exclude empty items when determining the result dtype. To retain the old behavior, exclude the empty entries before the concat operation.
  build_compressed_graph(self)
C:\Users\penny\git\Aequilibrae\aequilibrae\paths\graph.py:146: FutureWarning: The behavior of array concatenation with empty entries is deprecated. In a future version, this will no longer exclude empty items when determining the result dtype. To retain the old behavior, exclude the empty entries before the concat operation.
  build_compressed_graph(self)
C:\Users\penny\git\Aequilibrae\aequilibrae\paths\graph.py:146: FutureWarning: The behavior of array concatenation with empty entries is deprecated. In a future version, this will no longer exclude empty items when determining the result dtype. To retain the old behavior, exclude the 

In [3]:
# Get original flows:.
assignment.execute()
assign_df = assignment.results().reset_index(drop=False).fillna(0)
# SQUISH EXTRA DIMENSION FOR NOW - DEAL WITH THIS PROPERLY LATER ON!!!
matrix.matrix_view = np.squeeze(matrix.matrix_view, axis=2)

# Set the observed count volumes:
flow = lambda i: assign_df.loc[assign_df["link_id"] == i, "matrix_ab"].values[0]

In [4]:
assign_df

,link_id,matrix_ab,matrix_ba,matrix_tot,Congested_Time_AB,Congested_Time_BA,Congested_Time_Max,Delay_factor_AB,Delay_factor_BA,Delay_factor_Max,VOC_AB,VOC_BA,VOC_max,PCE_AB,PCE_BA,PCE_tot
0,1,5880.0,0.0,5880.0,6.002391,0.0,6.002391,1.000398,0.0,1.000398,0.227025,0.0,0.227025,5880.0,0.0,5880.0
1,2,10540.0,0.0,10540.0,4.024683,0.0,4.024683,1.006171,0.0,1.006171,0.450361,0.0,0.450361,10540.0,0.0,10540.0
2,3,6540.0,0.0,6540.0,6.003659,0.0,6.003659,1.000610,0.0,1.000610,0.252508,0.0,0.252508,6540.0,0.0,6540.0
3,4,6000.0,0.0,6000.0,6.608336,0.0,6.608336,1.321667,0.0,1.321667,1.210121,0.0,1.210121,6000.0,0.0,6000.0
4,5,9880.0,0.0,9880.0,4.019057,0.0,4.019057,1.004764,0.0,1.004764,0.422160,0.0,0.422160,9880.0,0.0,9880.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
71,72,11460.0,0.0,11460.0,20.558070,0.0,20.558070,5.139517,0.0,5.139517,2.292000,0.0,2.292000,11460.0,0.0,11460.0
72,73,11320.0,0.0,11320.0,9.405634,0.0,9.405634,4.702817,0.0,4.702817,2.229001,0.0,2.229001,11320.0,0.0,11320.0
73,74,11680.0,0.0,11680.0,20.619687,0.0,20.619687,5.154922,0.0,5.154922,2.294129,0.0,2.294129,11680.0,0.0,11680.0
74,75,8860.0,0.0,8860.0,7.868128,0.0,7.868128,2.622709,0.0,2.622709,1.813583,0.0,1.813583,8860.0,0.0,8860.0
